In [43]:
import torch
import torch.nn
import numpy as np

开始构建网络：

1.构建输入时间块

In [47]:
class InputBlockTime(torch.nn.Module):  #继承Module类的输入块类
    def __init__(self,time_channels,time_size,spatial_channels,spatial_size,pool_param = (1,2),dropout = 0.5):
        #这里写自己的魔法方法，这里的self指代的都是InputBlockTime这个类
        super(InputBlockTime, self).__init__() #超类：调用父类的初始化方法
        #输入16*1*64*256
        
        self.block  =torch.nn.Sequential( 
            #这是时间过滤器  
            #输入16*1*64*256
            torch.nn.Conv2d(in_channels=1,out_channels = time_channels,
                            kernel_size=time_size,padding = (0,time_size[1]//2),bias = False),
            #输出16*25*64*256,
            #计算公式为 H = [(H-2*P+K_P)/S] + 1
            
            #这是空间过滤器
            #输入16*25*64*256 
            torch.nn.Conv2d(in_channels=time_channels,groups = time_channels, #这里的groups分组卷积25
                            out_channels=spatial_channels,kernel_size = spatial_size),
            #输出16*50*1*256
            #批量归一化，防止数据过大
            torch.nn.BatchNorm2d(spatial_channels),
            #卷积中应该是不带激活函数的
            torch.nn.ELU(), #激活函数
            #最大池化
            torch.nn.MaxPool2d(kernel_size=pool_param,stride= pool_param),
            #输出 16*50*1*128
            #计算公式为：H = (H-K_H)/S +1

            #退却
            torch.nn.Dropout(dropout) 
            #用来防止过拟合，把一部分值置为很小接近于0
        )
        #这里创建自己的成员block
    def forward(self,x):
        x = self.block(x)
        return x
        

这里是分组卷积的图示，参数量减少为1/G
![](2023-02-19-17-22-27.png)

构建特征提取块：

这里我非常困惑的点是，为什么不能像之前那个块一样，直接用torch.Sequetial来构建

In [79]:
class FeatureBlock(torch.nn.Module):
    def __init__(self,in_channels,out_channels,conv_param = (1,15),pool_param = (1,2),dropout = 0.5,padding = (-1,-1,-1,-1)):
        super(FeatureBlock, self).__init__() #是因为torch.nn.Module本来就没有传入参数
        
        if(padding[0]==-1): #这一步我并不知道是什么意义，改写了padding元组，
            padding = (conv_param[1]//2,conv_param[1]//2,0,0)
            #应该是单边的添零

        #这一步是四周填0,分别是左右上下
        self.padding = torch.nn.ZeroPad2d(padding = padding)
        self.conv = torch.nn.Conv2d(in_channels = in_channels,
                                    out_channels=out_channels,
                                    kernel_size=conv_param,
                                    bias = False)
        #这一步不是很理解为什么num_features填这个值
        self.norm = torch.nn.BatchNorm2d(num_features=out_channels)
        self.elu = torch.nn.ELU()
        #这个池化，使得步长和卷积核相同是，输出尺寸为H/S
        self.pool = torch.nn.MaxPool2d(kernel_size = pool_param,
                                        stride = pool_param)
        self.drop_out = torch.nn.Dropout(dropout)
        
    def forward(self,x):
        #其实也就是，填充->卷积->批量归一化->激活->池化 
        #和前面那个模块感觉不出来区别
        x = self.padding(x)
        x = self.conv(x)
        x = self.norm(x)
        x = self.elu(x)
        x = self.pool(x)
        return self.drop_out(x)
        

        

构建深度卷积网络：

这里我很困惑的点是，为什么time_channels,time_size,spatial_channels,spatial_size,feature_pool_size,feature_channels_list这样子取值

In [92]:
class DeepConvNet(torch.nn.Module):
    def __init__(self,sample,class_num,time_channels,time_size,
                spatial_channels,spatial_size,
                feature_pool_size,feature_channels_list,
                dropout):
        super(DeepConvNet, self).__init__()
        #首先设置一个输入模块，用原来的输入模块，并传参
        self.input_block = InputBlockTime(time_channels=time_channels,time_size=time_size,
                                        spatial_channels=spatial_channels,spatial_size=spatial_size,
                                        pool_param=feature_pool_size,
                                        dropout = dropout)
        self.feature_block_list = torch.nn.Sequential()
        #以下通道赋值，是用来衔接输入模块，和特征提取模块
        pre_channels = spatial_channels
        for channel in feature_channels_list:
            self.feature_block_list.add_module(
                f"feature {channel}",  #第一个参数是名字，这里叫feature100
                FeatureBlock(in_channels=pre_channels,out_channels=channel,
                            pool_param=feature_pool_size,dropout=dropout)            
            )
            #这个是用来衔接两个特征提取模块
            pre_channels = channel

        #造一个数据来得到全连接层前数据的shape
        #不用手动计算了的技巧
        tmp_data = torch.Tensor(np.ones((1,1,64,sample),dtype=float))
        tmp_data = self.input_block(tmp_data)
        tmp_data = self.feature_block_list(tmp_data)
        #这里是把tmp_data压扁，重新视为size(0) * 自适应长度
        tmp_data = tmp_data.view(tmp_data.size(0),-1)
        self.classifer = torch.nn.Sequential(
            torch.nn.Linear(tmp_data.shape[1],class_num)
        )
    def forward(self,x):
        out2 = self.input_block(x) #输入模块
        x = self.feature_block_list(out2) #特征模块
        x = x.view(x.size(0),-1) #压缩成条
        x = self.classifer(x) #全连接层分类模块
        return torch.nn.functional.softmax(x,dim =1) #利用softmax,在1维归一化
        # return x
        

In [93]:

lhjNet= DeepConvNet(
        sample=256, class_num=2,
        time_channels=25, time_size=(1, 9),
        spatial_channels=50, spatial_size=(64, 1),
        feature_pool_size=(1, 3), feature_channels_list=[100, 200], dropout=0.5)
input =  np.ones((1,1,64,256)) 
input = torch.from_numpy(input).float()
output = lhjNet(input)

![](2023-02-19-17-53-13.png)

下面是拆解运算 

用来观察数据结构

In [86]:
a = np.ones((1,1,64,256)) 
a = torch.from_numpy(a).float()
#类的实例化，自动调用__init__方法
class DeepConvNet(torch.nn.Module):
    def __init__(self):
        super(DeepConvNet, self).__init__()
        self.input_block = InputBlockTime(time_channels=25,
                                            time_size = (1,9),
                                            spatial_channels=50,
                                            spatial_size=(64,1),
                                            pool_param=(1,2),
                                            dropout=0.5)
        self.feature_block_list= torch.nn.Sequential()
        pre_channels = 50
        feature_channels_list = [100,200]
        feature_pool_size = (1,3)
        dropout = 0.5

        for channel in feature_channels_list:
            self.feature_block_list.add_module(
                f"feature {channel}",  #第一个参数是名字，这里叫feature100
                FeatureBlock(in_channels=pre_channels,out_channels=channel,
                            pool_param=feature_pool_size,dropout=dropout)            
            )
            #这个是用来衔接两个特征提取模块
            pre_channels = channel


    def forward(self,x):
        out = self.input_block(x)
        out = self.feature_block_list(out)
        return out

net = DeepConvNet()




b = net(a) #自动调用farward方法
b = b.view(b.size(0),-1)
